In [15]:
import transformers
print(transformers.__version__)


4.49.0


In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, DistilBertConfig
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset


# Check for MPS availability and set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Clear MPS cache to free up memory
if device.type == "mps":
    torch.mps.empty_cache()

# Load and prepare data
df_master = pd.read_csv('master_data (1).csv')
df_converted = df_master.copy()
df_converted["Category"] = df_converted["Category"].map({"ham": 0, "spam": 1})

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_converted["Message"], df_converted["Category"], test_size=0.2, random_state=42, stratify=df_converted["Category"]
)

# Load tokenizer and encode data
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")  # Using a smaller model

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}  # Move tensors to MPS
        item["labels"] = torch.tensor(self.labels[idx]).to(device)  # Move labels to MPS
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels.tolist())
val_dataset = CustomDataset(val_encodings, val_labels.tolist())

# Configure model with higher dropout
config = DistilBertConfig.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    dropout=0.3,  # default is 0.1
    attention_dropout=0.3  # default is 0.1
)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", config=config)
model.to(device)  # Move model to MPS device

# Compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


Using device: mps


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
!pip install --upgrade transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [18]:
import transformers
print(transformers.__version__)


4.49.0


In [19]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# Trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


/Users/ayush/Documents/atif-ai/venv/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  if self.no_cuda:


In [20]:
# Train and evaluate
trainer.train()
eval_results = trainer.evaluate()
print(f"Performance on Eval set: {eval_results}")

# Save the model after training
save_directory = "./saved_modelv6"
trainer.save_model(save_directory)  # Save model, config, and tokenizer (if passed)
print(f"Model saved successfully to {save_directory}")

# Clear MPS cache after training
if device.type == "mps":
    torch.mps.empty_cache()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.144500,0.179022,0.952709,0.950579
2,0.040500,0.080112,0.977833,0.977683


Performance on Eval set: {'eval_loss': 0.08011189103126526, 'eval_accuracy': 0.9778325123152709, 'eval_f1': 0.9776829017898694, 'eval_runtime': 80.2219, 'eval_samples_per_second': 25.305, 'eval_steps_per_second': 0.399, 'epoch': 2.0}
Model saved successfully to ./saved_modelv6
